# AIM
In this notebook we train top layers of Deep CNNs (transfer learning)

## Setting Paths

In [1]:
import sys
from pathlib import Path
#Set root to be the main project folder
root = Path.cwd().parent
print(f"{root=}")

root=PosixPath('/home/gnacikm/Documents/GitHub/Melanoma')


In [2]:
data_path = Path(root/'data')
print(f"{data_path=}")
sav_models = Path(root/'saved_models')
print(f"{sav_models =}")

#Add location of py files to path so we can import
sys.path.insert(0,str(root))

data_path=PosixPath('/home/gnacikm/Documents/GitHub/Melanoma/data')
sav_models =PosixPath('/home/gnacikm/Documents/GitHub/Melanoma/saved_models')


## LIBRARIES / PACKAGES

In [3]:
import tensorflow as tf

2021-08-06 09:50:25.705528: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


## Importing Models

In [4]:
from models.deepnet import DeepNet

### Importing Methods from Utils

In [5]:
from utils.tools import ImageGenerator, FlowFromDir

### Checking if GPU has been detected

In [6]:
device_name = tf.test.gpu_device_name()
if device_name == '':
    raise SystemError("GPU not found")
else:
    print(f"GPU {device_name} found")

GPU /device:GPU:0 found


2021-08-06 08:49:47.706900: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-08-06 08:49:47.710769: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-08-06 08:49:47.766718: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-06 08:49:47.767032: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:02:00.0 name: GeForce GTX 980 computeCapability: 5.2
coreClock: 1.329GHz coreCount: 16 deviceMemorySize: 3.95GiB deviceMemoryBandwidth: 208.91GiB/s
2021-08-06 08:49

## Processing Images

#### Setting paths

In [7]:
dir_with_zip = data_path
dir_with_data = f"{dir_with_zip}/DermMel" 
path_train = f"{dir_with_data}/train_sep/"
path_val = f"{dir_with_data}/valid/"
path_test = f"{dir_with_data}/test/"

#### Using ImageDataGenerator
See https://keras.io/api/preprocessing/image/

In [8]:
data_train = ImageGenerator()
data_val = ImageGenerator()
data_test = ImageGenerator()

In [9]:
train_generator = FlowFromDir(
    data_train,
    path_train,
    target_size=(160, 160),
    shuffle=True)
val_generator = FlowFromDir(
    data_val,
    path_val,
    target_size=(160, 160),
    shuffle=True)
test_generator = FlowFromDir(
    data_test,
    path_test,
    target_size=(160, 160),
    batch_size=1)

Found 10683 images belonging to 2 classes.
Found 3562 images belonging to 2 classes.
Found 3561 images belonging to 2 classes.


### Input and output shapes

In [10]:
input_shape = train_generator[0][0].shape[1:]
input_shape

(160, 160, 3)

In [11]:
num_of_classes = train_generator[0][1][0].shape[0]
num_of_classes 

2

# MobileNetV2 - Initiating the model

### Setting model parameters

In [12]:
EPOCHS = 15
inputs = tf.keras.Input(input_shape)
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-04)

In [13]:
mobilenetv2_model = tf.keras.applications.MobileNetV2(input_shape=input_shape,
                                                    include_top=False,
                                                    weights='imagenet',
                                                    pooling=None, 
                                                    classifier_activation="softmax"
                                                )

2021-08-06 08:49:56.618993: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-06 08:49:56.619358: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:02:00.0 name: GeForce GTX 980 computeCapability: 5.2
coreClock: 1.329GHz coreCount: 16 deviceMemorySize: 3.95GiB deviceMemoryBandwidth: 208.91GiB/s
2021-08-06 08:49:56.619449: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-06 08:49:56.619806: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-06 08:49:56.620090: I tensorflow/core/common_runtim

In [14]:
MobModel = DeepNet(mobilenetv2_model, num_of_classes)
MobModel.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=["accuracy"]
)

### Callbacks

In [16]:
# ModelCheckpoint
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    sav_models/"weights/mobnet.h5", 
    monitor = "val_accuracy", 
    verbose = 2, 
    save_weights_only=True,
    save_best_only = True,
    mode = 'max'
)

### Fitting the model

In [22]:
history_mob = MobModel.fit(train_generator, steps_per_epoch=train_generator.n//train_generator.batch_size, 
                    epochs=EPOCHS, 
                    validation_data=val_generator, 
                    validation_steps=val_generator.n//val_generator.batch_size,
                    callbacks=[checkpoint])

2021-08-05 14:37:34.430798: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudnn.so.8
2021-08-05 14:37:34.877954: I tensorflow/stream_executor/cuda/cuda_dnn.cc:359] Loaded cuDNN version 8202
2021-08-05 14:37:35.211361: E tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2021-08-05 14:37:35.211386: W tensorflow/stream_executor/gpu/asm_compiler.cc:56] Couldn't invoke ptxas --version
2021-08-05 14:37:35.211785: E tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2021-08-05 14:37:35.211862: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2021-08-05 14:37:35.239227: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully ope

Epoch 1/15


2021-08-05 14:37:40.306741: W tensorflow/core/common_runtime/bfc_allocator.cc:271] Allocator (GPU_0_bfc) ran out of memory trying to allocate 376.59MiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2021-08-05 14:37:40.316962: W tensorflow/core/common_runtime/bfc_allocator.cc:271] Allocator (GPU_0_bfc) ran out of memory trying to allocate 489.09MiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2021-08-05 14:37:40.342671: W tensorflow/core/common_runtime/bfc_allocator.cc:271] Allocator (GPU_0_bfc) ran out of memory trying to allocate 827.17MiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2021-08-05 14:37:40.383598: W tensorflow/core/common_runtime/bfc_allocator.cc:271]

166/166 [==============================] - 162s 944ms/step - loss: 0.3092 - accuracy: 0.8739 - val_loss: 0.6272 - val_accuracy: 0.5361

Epoch 00001: val_accuracy improved from -inf to 0.53608, saving model to weights/mobnet.h5
Epoch 2/15
166/166 [==============================] - 149s 896ms/step - loss: 0.2080 - accuracy: 0.9201 - val_loss: 0.1877 - val_accuracy: 0.9270

Epoch 00002: val_accuracy improved from 0.53608 to 0.92699, saving model to weights/mobnet.h5
Epoch 3/15
166/166 [==============================] - 146s 874ms/step - loss: 0.1791 - accuracy: 0.9293 - val_loss: 0.1599 - val_accuracy: 0.9398

Epoch 00003: val_accuracy improved from 0.92699 to 0.93977, saving model to weights/mobnet.h5
Epoch 4/15
166/166 [==============================] - 181s 1s/step - loss: 0.1636 - accuracy: 0.9376 - val_loss: 0.1392 - val_accuracy: 0.9426

Epoch 00004: val_accuracy improved from 0.93977 to 0.94261, saving model to weights/mobnet.h5
Epoch 5/15
166/166 [==============================] -

### Model Summary

In [15]:
MobModel.call(inputs)
MobModel.built = True 
MobModel.load_weights(sav_models/"weights/mobnet.h5")
MobModel.summary()
MobModel.summary()

Model: "deep_net"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_160 (Functi (None, 5, 5, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 120)               153720    
_________________________________________________________________
dense_1 (Dense)              (None, 84)                10164     
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 170       
_________________________________________________________________
dropout (Dropout)            multiple                  0 (unused)
Total params: 2,422,038
Trainable params: 2,387,926
Non-trainable params: 34,112
___________________________________________

## Prediction accuracy on test dataset

In [16]:
test_generator.reset()
MobModel.evaluate(test_generator, steps=test_generator.n//test_generator.batch_size)

2021-08-06 08:50:12.322697: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-08-06 08:50:12.342598: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 3399905000 Hz
2021-08-06 08:50:13.299782: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudnn.so.8
2021-08-06 08:50:13.613235: I tensorflow/stream_executor/cuda/cuda_dnn.cc:359] Loaded cuDNN version 8202
2021-08-06 08:50:13.856646: E tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2021-08-06 08:50:13.856671: W tensorflow/stream_executor/gpu/asm_compiler.cc:56] Couldn't invoke ptxas --version
2021-08-06 08:50:13.857250: E tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2021-08-06 08:50:13.857312: W tensorflow/stream_executor/gpu/redzone_allocator.cc:31

3561/3561 [==============================] - 37s 10ms/step - loss: 0.1342 - accuracy: 0.9461


[0.13421431183815002, 0.9460825324058533]

# VGG19 - Initiating the model

In [11]:
EPOCHS = 15
inputs = tf.keras.Input(input_shape)
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-04)

In [14]:
vgg19_model = tf.keras.applications.vgg19.VGG19(
    include_top=False,
    weights='imagenet',
    input_shape=input_shape,
    pooling=None, 
    classifier_activation='softmax'
)

In [15]:
Vgg19Model = DeepNet(vgg19_model, num_of_classes)
Vgg19Model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=["accuracy"]
)

## Callbacks

In [16]:
# ModelCheckpoint
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    sav_models/"weights/vgg19.h5", 
    monitor = "val_accuracy", 
    verbose = 2, 
    save_weights_only=True,
    save_best_only = True,
    mode = 'max'
)

## Fitting the model

In [17]:
history_vgg = Vgg19Model.fit(train_generator, steps_per_epoch=train_generator.n//train_generator.batch_size, 
                    epochs=EPOCHS, 
                    validation_data=val_generator, 
                    validation_steps=val_generator.n//val_generator.batch_size,
                    callbacks=[checkpoint])

2021-08-05 15:26:16.120282: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudnn.so.8
2021-08-05 15:26:16.446528: I tensorflow/stream_executor/cuda/cuda_dnn.cc:359] Loaded cuDNN version 8202
2021-08-05 15:26:16.713274: E tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2021-08-05 15:26:16.713310: W tensorflow/stream_executor/gpu/asm_compiler.cc:56] Couldn't invoke ptxas --version
2021-08-05 15:26:16.713774: E tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2021-08-05 15:26:16.713896: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2021-08-05 15:26:16.731858: W tensorflow/core/common_runtime/bfc_allocator.cc:271] Allocator (GPU_0_bfc) ran

Epoch 1/15


2021-08-05 15:26:20.325484: W tensorflow/core/common_runtime/bfc_allocator.cc:271] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.03GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2021-08-05 15:26:20.491138: W tensorflow/core/common_runtime/bfc_allocator.cc:271] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.53GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2021-08-05 15:26:22.869758: W tensorflow/core/kernels/gpu_utils.cc:49] Failed to allocate memory for convolution redzone checking; skipping this check. This is benign and only means that we won't check cudnn for out-of-bounds reads and writes. This message will only be printed once.


166/166 [==============================] - 269s 2s/step - loss: 0.4903 - accuracy: 0.7667 - val_loss: 0.4361 - val_accuracy: 0.8097

Epoch 00001: val_accuracy improved from -inf to 0.80966, saving model to weights/vgg19.h5
Epoch 2/15
166/166 [==============================] - 223s 1s/step - loss: 0.3825 - accuracy: 0.8424 - val_loss: 0.3824 - val_accuracy: 0.8264

Epoch 00002: val_accuracy improved from 0.80966 to 0.82642, saving model to weights/vgg19.h5
Epoch 3/15
166/166 [==============================] - 213s 1s/step - loss: 0.3635 - accuracy: 0.8450 - val_loss: 0.4225 - val_accuracy: 0.8236

Epoch 00003: val_accuracy did not improve from 0.82642
Epoch 4/15
166/166 [==============================] - 210s 1s/step - loss: 0.3124 - accuracy: 0.8732 - val_loss: 0.2668 - val_accuracy: 0.8878

Epoch 00004: val_accuracy improved from 0.82642 to 0.88778, saving model to weights/vgg19.h5
Epoch 5/15
166/166 [==============================] - 208s 1s/step - loss: 0.2947 - accuracy: 0.8800 - v

### Model Summary

In [19]:
Vgg19Model.call(inputs)
Vgg19Model.built = True
Vgg19Model.load_weights(sav_models/"weights/vgg19.h5")
Vgg19Model.summary()

Model: "deep_net_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Functional)           (None, 5, 5, 512)         20024384  
_________________________________________________________________
global_average_pooling2d_1 ( multiple                  0         
_________________________________________________________________
dense_3 (Dense)              multiple                  61560     
_________________________________________________________________
dense_4 (Dense)              multiple                  10164     
_________________________________________________________________
dense_5 (Dense)              multiple                  170       
_________________________________________________________________
dropout_1 (Dropout)          multiple                  0         
Total params: 20,096,278
Trainable params: 20,096,278
Non-trainable params: 0
____________________________________________

## Prediction accuracy on test dataset

In [22]:
test_generator.reset()
Vgg19Model.evaluate(test_generator, steps=test_generator.n//test_generator.batch_size)

3561/3561 [==============================] - 43s 12ms/step - loss: 0.1683 - accuracy: 0.9399


[0.16827549040317535, 0.939904510974884]

# ResNet 50V2 - Initiating the model

In [28]:
EPOCHS = 15
inputs = tf.keras.Input(input_shape)
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-04)

In [29]:
resnet_model = tf.keras.applications.ResNet50V2(
    include_top=False, 
    weights="imagenet",
    input_shape=input_shape,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
)

In [30]:
ResNetModel = DeepNet(resnet_model, num_of_classes)
ResNetModel.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=["accuracy"]
)

## Callbacks

In [31]:
# ModelCheckpoint
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    sav_models/"weights/resnet.h5", 
    monitor = "val_accuracy", 
    verbose = 2, 
    save_weights_only=True,
    save_best_only = True,
    mode = 'max'
)

## Fitting the model

In [32]:
history_res = ResNetModel.fit(train_generator, steps_per_epoch=train_generator.n//train_generator.batch_size, 
                    epochs=EPOCHS, 
                    validation_data=val_generator, 
                    validation_steps=val_generator.n//val_generator.batch_size,
                    callbacks=[checkpoint])

2021-08-05 18:21:35.891103: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudnn.so.8
2021-08-05 18:21:36.138894: I tensorflow/stream_executor/cuda/cuda_dnn.cc:359] Loaded cuDNN version 8202
2021-08-05 18:21:36.359162: E tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2021-08-05 18:21:36.359201: W tensorflow/stream_executor/gpu/asm_compiler.cc:56] Couldn't invoke ptxas --version
2021-08-05 18:21:36.359918: E tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2021-08-05 18:21:36.360105: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2021-08-05 18:21:36.376098: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully ope

Epoch 1/15


2021-08-05 18:21:40.161387: W tensorflow/core/common_runtime/bfc_allocator.cc:271] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.29GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2021-08-05 18:21:40.172937: W tensorflow/core/common_runtime/bfc_allocator.cc:271] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.34GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2021-08-05 18:21:40.289055: W tensorflow/core/common_runtime/bfc_allocator.cc:271] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.27GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2021-08-05 18:21:40.295478: W tensorflow/core/common_runtime/bfc_allocator.cc:271] Alloc

121/667 [====>.........................] - ETA: 1:11 - loss: 0.5710 - accuracy: 0.7138

2021-08-05 18:21:56.900762: W tensorflow/core/common_runtime/bfc_allocator.cc:271] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.28GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2021-08-05 18:21:56.910151: W tensorflow/core/common_runtime/bfc_allocator.cc:271] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.32GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2021-08-05 18:21:56.922276: W tensorflow/core/common_runtime/bfc_allocator.cc:271] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.27GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2021-08-05 18:21:56.928689: W tensorflow/core/common_runtime/bfc_allocator.cc:271] Alloc

667/667 [==============================] - 116s 168ms/step - loss: 0.4297 - accuracy: 0.8065 - val_loss: 0.3858 - val_accuracy: 0.8083

Epoch 00001: val_accuracy improved from -inf to 0.80828, saving model to /home/gnacikm/Documents/GitHub/Melanoma/saved_models/weights/resnet.h5
Epoch 2/15
667/667 [==============================] - 110s 165ms/step - loss: 0.3275 - accuracy: 0.8645 - val_loss: 0.2966 - val_accuracy: 0.8722

Epoch 00002: val_accuracy improved from 0.80828 to 0.87218, saving model to /home/gnacikm/Documents/GitHub/Melanoma/saved_models/weights/resnet.h5
Epoch 3/15
667/667 [==============================] - 110s 165ms/step - loss: 0.2944 - accuracy: 0.8784 - val_loss: 0.2277 - val_accuracy: 0.9060

Epoch 00003: val_accuracy improved from 0.87218 to 0.90597, saving model to /home/gnacikm/Documents/GitHub/Melanoma/saved_models/weights/resnet.h5
Epoch 4/15
667/667 [==============================] - 112s 167ms/step - loss: 0.2709 - accuracy: 0.8911 - val_loss: 0.2833 - val_acc

### Model Summary

In [37]:
ResNetModel.call(inputs)
ResNetModel.built = True 
ResNetModel.load_weights(sav_models/"weights/resnet.h5")
ResNetModel.summary()

Model: "deep_net_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50v2 (Functional)      (None, 5, 5, 2048)        23564800  
_________________________________________________________________
global_average_pooling2d_3 ( multiple                  0         
_________________________________________________________________
dense_9 (Dense)              multiple                  245880    
_________________________________________________________________
dense_10 (Dense)             multiple                  10164     
_________________________________________________________________
dense_11 (Dense)             multiple                  170       
_________________________________________________________________
dropout_3 (Dropout)          multiple                  0         
Total params: 23,821,014
Trainable params: 23,775,574
Non-trainable params: 45,440
_______________________________________

## Prediction accuracy on test dataset

In [40]:
test_generator.reset()
ResNetModel.evaluate(test_generator, steps=test_generator.n//test_generator.batch_size)

3561/3561 [==============================] - 41s 12ms/step - loss: 0.2095 - accuracy: 0.9096


[0.20945307612419128, 0.9095759391784668]

# Xception - Initiating the model

In [41]:
EPOCHS = 15
inputs = tf.keras.Input(input_shape)
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-04)

In [42]:
xception_model = tf.keras.applications.xception.Xception(
    include_top=False,
    weights='imagenet',
    input_shape=input_shape,
    pooling=None, 
    classifier_activation='softmax'
)

In [43]:
XcpModel = DeepNet(xception_model, num_of_classes)
XcpModel.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=["accuracy"]
)

### Callbacks

In [44]:
# ModelCheckpoint
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    sav_models/"weights/xception.h5", 
    monitor = "val_accuracy", 
    verbose = 2, 
    save_weights_only=True,
    save_best_only = True,
    mode = 'max'
)

## Fitting the model

In [45]:
history_xcp = XcpModel.fit(train_generator, steps_per_epoch=train_generator.n//train_generator.batch_size, 
                    epochs=EPOCHS, 
                    validation_data=val_generator, 
                    validation_steps=val_generator.n//val_generator.batch_size,
                    callbacks=[checkpoint])

Epoch 1/15
667/667 [==============================] - 189s 279ms/step - loss: 0.2950 - accuracy: 0.8817 - val_loss: 0.1576 - val_accuracy: 0.9381

Epoch 00001: val_accuracy improved from -inf to 0.93806, saving model to /home/gnacikm/Documents/GitHub/Melanoma/saved_models/weights/xception.h5
Epoch 2/15
667/667 [==============================] - 182s 273ms/step - loss: 0.2821 - accuracy: 0.8928 - val_loss: 0.1629 - val_accuracy: 0.9372

Epoch 00002: val_accuracy did not improve from 0.93806
Epoch 3/15
667/667 [==============================] - 181s 271ms/step - loss: 0.1739 - accuracy: 0.9307 - val_loss: 0.1509 - val_accuracy: 0.9358

Epoch 00003: val_accuracy did not improve from 0.93806
Epoch 4/15
667/667 [==============================] - 186s 279ms/step - loss: 0.1655 - accuracy: 0.9352 - val_loss: 0.1537 - val_accuracy: 0.9395

Epoch 00004: val_accuracy improved from 0.93806 to 0.93947, saving model to /home/gnacikm/Documents/GitHub/Melanoma/saved_models/weights/xception.h5
Epoch 5

### Model Summary

In [46]:
XcpModel.call(inputs)
XcpModel.built = True 
XcpModel.load_weights(sav_models/"weights/xception.h5")
XcpModel.summary()

Model: "deep_net_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Functional)        (None, 5, 5, 2048)        20861480  
_________________________________________________________________
global_average_pooling2d_4 ( multiple                  0         
_________________________________________________________________
dense_12 (Dense)             multiple                  245880    
_________________________________________________________________
dense_13 (Dense)             multiple                  10164     
_________________________________________________________________
dense_14 (Dense)             multiple                  170       
_________________________________________________________________
dropout_4 (Dropout)          multiple                  0         
Total params: 21,117,694
Trainable params: 21,063,166
Non-trainable params: 54,528
_______________________________________

## Prediction accuracy on test dataset

In [48]:
test_generator.reset()
XcpModel.evaluate(test_generator, steps=test_generator.n//test_generator.batch_size)

3561/3561 [==============================] - 41s 11ms/step - loss: 0.1243 - accuracy: 0.9523


[0.12426680326461792, 0.9522606134414673]